In [3]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import PIL
from google.colab.patches import cv2_imshow
import gspread
import torch
import torchvision
import torchvision.transforms as transforms
import sklearn

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
drive.mount("/content/gdrive", force_remount=True)

In [4]:
NoFind = 0
Covid = 1
Atelectasis = 3
Cardiomegaly = 4
Effusion = 5
Infiltration = 6
MASS = 7
Nodule = 8
Pneumonia = 9
Pneumothorax = 10
Consolidation = 11
Edema = 12
Emphysema, Fibrosis, Pleural, Hernia, Tuberculosis, Opacity, Other = 13, 14, 15, 16, 17, 18, 19
testing_path = '/content/gdrive/MyDrive/DiseaseDetection/Testing/Images/'

tb_path_pos = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Tuberculosis/'
tb_path_neg = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Normal/'
tb_data = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Tuberculosis.csv'

cov1_path_cov = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/COVID/images/'
cov1_path_vir = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Viral Pneumonia/images/'
cov1_path_opac = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Lung_Opacity/images/'
cov1_path_neg = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Normal/images/'
cov1_data = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/data.csv'

cov2_images = '/content/gdrive/MyDrive/DiseaseDetection/covid-chestxray-dataset-master/images/'
cov2_data = '/content/gdrive/MyDrive/DiseaseDetection/covid-chestxray-dataset-master/Data.csv'

dd_paths = []
for i in range(1, 13):
  x = '/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_0'
  if i < 10:
    x += '0'
  x += str(i) + '/images/'
  dd_paths.append(x)

dd_data = '/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/data.csv'

In [5]:
tb_df = pd.read_csv(tb_data)
cov1_df = pd.read_csv(cov1_data)
cov2_df = pd.read_csv(cov2_data)
dd_df = pd.read_csv(dd_data)

tb_df.rename(columns = {"FILE NAME": "Name"}, inplace = True)
cov1_df.rename(columns = {"FILE NAME": "Name"}, inplace = True)
cov2_df.rename(columns = {"filename": "Name", 'finding': 'Diagnosis'}, inplace = True)
dd_df.rename(columns = {"Image Index": "Name", 'Finding Labels': 'Diagnosis'}, inplace = True)

def split(x):
  if "|" in x:
    return x.split('|')
  elif "/" in x:
    return x.split('/')
  else:
    if type(x) != list:
      return [x]
    else:
      return x
def organize(x):
  for i in range(len(x)):
    if x[i].lower() == "covid" or x[i].lower() == "covid-19":
      x[i] = Covid
    elif x[i] == "None" or x[i] == "No Finding":
      x[i] = NoFind
    elif x[i] == "Viral":
      x[i] = Pneumonia
    elif x[i] == "Atelectasis":
      x[i] = Atelectasis
    elif x[i] == "Cardiomegaly":
      x[i] = Cardiomegaly
    elif x[i] == "Effusion":
      x[i] = Effusion
    elif x[i] == "Infiltration":
      x[i] = Infiltration
    elif x[i] == "Mass":
      x[i] = MASS
    elif x[i] == "Nodule":
      x[i] = Nodule
    elif x[i] == "Pneumonia":
      x[i] = Pneumonia
    elif x[i] == "Pneumothorax":
      x[i] = Pneumothorax
    elif x[i] == "Consolidation":
      x[i] = Consolidation
    elif x[i] == "Edema":
      x[i] = Edema
    elif x[i] == "Emphysema":
      x[i] = Emphysema
    elif x[i] == "Fibrosis":
      x[i] = Fibrosis
    elif x[i] == "Pleural_Thickening":
      x[i] = Pleural
    elif x[i] == "Hernia":
      x[i] = Hernia
    elif x[i] == "Tuberculosis":
      x[i] = Tuberculosis
    elif x[i] == "Opacity":
      x[i] = Opacity
    else:
      x[i] = Other
  return x
def cov1_modify(x):
  if "NORMAL" in x:
    return x.capitalize() + '.png'
  return x + '.png'

def tb_modify(x):
  if "Normal" in x:
    return "Normal TB-" + x.split("-")[1] + '.png'
  return x + ".png"
cov1_df["Name"] = cov1_df["Name"].apply(cov1_modify)
tb_df["Name"] = tb_df["Name"].apply(tb_modify)


df = pd.concat([dd_df, cov2_df, cov1_df, tb_df], ignore_index=True)
df["Diagnosis"] = df["Diagnosis"].apply(split)
df["Diagnosis"] = df["Diagnosis"].apply(organize)
df.rename(columns = {"Name": "Image Name", 'Diagnosis': 'Classification'}, inplace = True)

In [6]:
temp = df["Classification"].copy()
labels = {df["Image Name"][i]: temp[i][0] for i in range(len(df))}

In [2]:
X_test = []

In [7]:
for i in os.listdir(tb_path_pos): 
  curr = Image.open(tb_path_pos + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  curr = np.stack((curr,)*3)
  curr = np.transpose(curr, (1, 2, 0))
  curr = PIL.Image.fromarray(curr)
  X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [8]:
for i in os.listdir(tb_path_neg):
  curr = Image.open(tb_path_neg + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  curr = np.stack((curr,)*3)
  curr = np.transpose(curr, (1, 2, 0))
  curr = PIL.Image.fromarray(curr)
  X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [9]:
for i in os.listdir(cov1_path_neg):
  if "(" not in i:
    curr = Image.open(cov1_path_neg + i)
    curr = curr.resize((32,32))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [10]:
for i in os.listdir(cov1_path_opac):
  curr = Image.open(cov1_path_opac + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  curr = np.stack((curr,)*3)
  curr = np.transpose(curr, (1, 2, 0))
  curr = PIL.Image.fromarray(curr)
  X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [11]:
for i in os.listdir(cov1_path_vir):
  curr = Image.open(cov1_path_vir + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  curr = np.stack((curr,)*3)
  curr = np.transpose(curr, (1, 2, 0))
  curr = PIL.Image.fromarray(curr)
  X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [12]:
for i in os.listdir(cov1_path_cov):
  if "(" not in i:
    curr = Image.open(cov1_path_cov + i)
    curr = curr.resize((32,32))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
import time
start = time.time()
for i in dd_paths:
  print(i, time.time() - start)
  for j in os.listdir(i):
 
    curr = Image.open(i + j)
    curr = curr.resize((32,32))
    curr = curr.convert("L")
    curr = np.stack((curr,)*3)
    curr = np.transpose(curr, (1, 2, 0))
    curr = PIL.Image.fromarray(curr)
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[j]))
end = time.time()
print(end - start)

/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_001/images/ 0.00010323524475097656
/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_002/images/ 231.2717523574829


In [ ]:
for i in os.listdir(cov2_images):
  curr = Image.open(cov2_images + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  curr = np.stack((curr,)*3)
  curr = np.transpose(curr, (1, 2, 0))
  curr = PIL.Image.fromarray(curr)
  if i in labels:
    X_test.append(((np.transpose(np.asarray(curr), (2, 1, 0)) / 255), labels[i]))

In [ ]:
print(len(X_test))

138331


In [ ]:
import random
X_test2 = []
for i in range(len(X_test)):
  if X_test[i][1] == 0 and random.randint(1, 5) != 1:
    continue
  X_test2.append(X_test[i])

In [ ]:
c = []
for i in X_test2:
  c.append(i[1])
print(pd.Series(c).value_counts())

In [ ]:
def convert_to_imshow_format(image):
    # image = image / 2 + 0.5
    # image = image.numpy()
    return image.transpose(2,1,0) * 255

In [ ]:
for i in X_test:
  cv2_imshow(convert_to_imshow_format(i[0]))

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split([i[0] for i in X_test2], [i[1] for i in X_test2], test_size = 0.2)

In [ ]:
new_x_train = []
for i in range(len(x_train)):
  new_x_train.append([x_train[i], y_train[i]])
new_x_test = []
for i in range(len(x_test)):
  new_x_test.append([x_test[i], y_test[i]])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
trainloader = torch.utils.data.DataLoader(new_x_train, batch_size=20, shuffle=True)
testloader = torch.utils.data.DataLoader(new_x_test, batch_size=len(new_x_test), shuffle=True)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
net.to(device)

In [ ]:
epochs = 10

# print(torch.cuda.get_device_name(0))
# if torch.cuda.is_available():
#     net = net.cuda()
#     criterion = criterion.cuda()

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # inputs = []
        # forward + backward + optimize
        # for j in X_test:
        #   inputs.append(X_test[j])
        outputs = net(inputs.float())
        # outputs = torch.Tensor(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0
print('Finished Training.')

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)

In [ ]:
outputs = net(images.float())

In [ ]:
sm = nn.Softmax(dim=1)
sm_outputs = sm(outputs)
# print(sm_outputs)

In [ ]:
dataiter = iter(torch.utils.data.DataLoader(new_x_train, batch_size=len(new_x_train), shuffle=True))
images, labels = dataiter.next()
images = images.to(device)
outputs = net(images.float())
sm = nn.Softmax(dim=1)
sm_outputs = sm(outputs)

In [ ]:
def get_acc(in_list, labels):
  correct = 0
  for j in range(len(in_list)):
    max = -1
    max_index = -1
    for i, val in enumerate(in_list[j]):
      if val > max:
        max = val
        max_index = i
    if max_index == labels[j]:
      # print('Got Right: ' + str(labels[j]))
      correct += 1
    else:
      print('Got Wrong: ' + str(labels[j]) + '(Correct) ' + str(max_index) + '(Guess)')
  return correct / len(labels)

In [ ]:
def get_acc_2(in_list, labels):
  count = 0
  for j in range(len(in_list)):
    if labels[j] in sorted(range(len(in_list[j])), key=lambda i: in_list[j][i], reverse=True)[:3]:
      count += 1
  return count / len(labels)

In [ ]:
get_acc_2(sm_outputs, labels)

In [ ]:
get_acc(sm_outputs, labels)